# Sexto notebook

En este penultimo notebook se  analiza el desempeño real del modelo entrenado para decidir
si el bajo rendimiento (si existe) se debe a:
- umbrales mal configurados
- clases confundidas
- pocos datos
- o necesidad de más entrenamiento

Este notebook:
- Evalúa el modelo con distintos thresholds
- Analiza predicciones incorrectas
- Resume errores por clase
- Ayuda a decidir si cambiar a dataset completo


In [14]:
# --- IMPORTS ---
from pathlib import Path
import json
from collections import defaultdict, Counter


In [15]:
# --- RUTAS DEL PROYECTO ---
#rutas creadas en los previos notebooks
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

MODELS_DIR = PROJECT_ROOT / "models"
YOLO_MODEL_PATH = MODELS_DIR / "yolo_best.pt"

YOLO_DATASET_DIR = PROJECT_ROOT / "data" / "processed" / "yolo_dataset"
DATA_YAML_PATH = YOLO_DATASET_DIR / "data.yaml"

ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
CONFIG_SNAPSHOT_PATH = ARTIFACTS_DIR / "config_snapshot.json"

print("Modelo:", YOLO_MODEL_PATH)
print("Dataset:", DATA_YAML_PATH)


Modelo: c:\Users\Johnny\Desktop\IA\models\yolo_best.pt
Dataset: c:\Users\Johnny\Desktop\IA\data\processed\yolo_dataset\data.yaml


In [16]:
# --- VALIDACIONES ---
#es fundamental que ya exista el modelo entrenado y el yaml por eso de debe validar que existen y son consistentes
def assert_exists(p: Path, desc: str) -> None:
    if not p.exists():
        raise FileNotFoundError(f"Falta {desc}: {p}")

assert_exists(YOLO_MODEL_PATH, "modelo entrenado")
assert_exists(DATA_YAML_PATH, "data.yaml")

print("Validaciones OK.")


Validaciones OK.


In [17]:
# --- CARGA DE CONFIGURACIÓN ---
#carga del archivo .json del modelo yolo
with open(CONFIG_SNAPSHOT_PATH, "r", encoding="utf-8") as f:
    cfg = json.load(f)

cfg


{'target_classes': ['car', 'airplane', 'truck'],
 'max_images_per_class_train': 1200,
 'max_images_per_class_val': 250,
 'img_size': 640,
 'batch_size': 16,
 'epochs': 20,
 'seed': 42,
 'conf_threshold': 0.25,
 'iou_threshold': 0.5}

1. Validacion del modelo 

In [18]:
# --- IMPORTAR YOLO ---
from ultralytics import YOLO

model = YOLO(str(YOLO_MODEL_PATH))


In [19]:
# --- EVALUACIÓN BASE (THRESHOLDS POR DEFECTO) ---
#Validacion de que si se haya cumpleido los parametros inciiales del rpimer notebook con respetco al consumo de datos, tamanano de imagenes y batch a usarse
base_metrics = model.val(
    data=str(DATA_YAML_PATH),
    imgsz=cfg["img_size"],
    conf=cfg["conf_threshold"],
    iou=cfg["iou_threshold"],
    batch=cfg["batch_size"]
)

print("Evaluación base completada.")


Ultralytics 8.4.8  Python-3.12.10 torch-2.9.1+cpu CPU (Intel Core 7 150U)
Model summary (fused): 73 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 6.71.2 MB/s, size: 152.0 KB)
val: Scanning C:\Users\Johnny\Desktop\IA\data\processed\yolo_dataset\labels\val.cache... 550 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 550/550  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 35/35 1.8s/it 1:041.8sss
                   all        550       1895      0.571      0.485      0.536      0.361
                   car        378       1337       0.52        0.5      0.516      0.332
              airplane         97        143      0.808      0.587      0.747      0.522
                 truck        250        415      0.385      0.369      0.344      0.229
Speed: 1.5ms preprocess, 81.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to C:\Users\Johnny\runs\

2. Evaluación con distintos thresholds

Aquí probamos diferentes valores de `conf` para ver
cómo cambian precision y recall sin reentrenar el modelo.

In [20]:
# --- BARRIDO DE CONFIDENCE THRESHOLD ---
conf_values = [0.1, 0.25, 0.4, 0.6]
conf_results = {}

for conf in conf_values:
    results = model.val(
        data=str(DATA_YAML_PATH),
        imgsz=cfg["img_size"],
        conf=conf,
        iou=cfg["iou_threshold"],
        batch=cfg["batch_size"],
        verbose=False
    )
    conf_results[conf] = {
        "precision": float(results.results_dict.get("metrics/precision(B)", 0)),
        "recall": float(results.results_dict.get("metrics/recall(B)", 0)),
        "mAP50": float(results.results_dict.get("metrics/mAP50(B)", 0))
    }

conf_results


Ultralytics 8.4.8  Python-3.12.10 torch-2.9.1+cpu CPU (Intel Core 7 150U)
val: Fast image access  (ping: 0.10.0 ms, read: 287.081.6 MB/s, size: 159.9 KB)
val: Scanning C:\Users\Johnny\Desktop\IA\data\processed\yolo_dataset\labels\val.cache... 550 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 550/550  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 35/35 1.3s/it 46.7s1.4ss
                   all        550       1895      0.571      0.485       0.52      0.337
Speed: 0.8ms preprocess, 72.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to C:\Users\Johnny\runs\detect\val9
Ultralytics 8.4.8  Python-3.12.10 torch-2.9.1+cpu CPU (Intel Core 7 150U)
val: Fast image access  (ping: 0.00.0 ms, read: 558.1108.0 MB/s, size: 149.9 KB)
val: Scanning C:\Users\Johnny\Desktop\IA\data\processed\yolo_dataset\labels\val.cache... 550 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 550/550  0.0s
                 C

{0.1: {'precision': 0.5708370741630303,
  'recall': 0.4854870859286427,
  'mAP50': 0.5202825158562202},
 0.25: {'precision': 0.5708370741630303,
  'recall': 0.4854870859286427,
  'mAP50': 0.5359127689191482},
 0.4: {'precision': 0.5903571591106672,
  'recall': 0.46776135616102815,
  'mAP50': 0.5348483211826959},
 0.6: {'precision': 0.7271852662841035,
  'recall': 0.3485598360511626,
  'mAP50': 0.5361896936549514}}

## Interpretación rápida

- Si al bajar `conf` mejora mucho el recall → faltan detecciones
- Si al subir `conf` mejora precision → hay muchos falsos positivos
- Si todo es bajo → el problema es **datos o entrenamiento**, no thresholds


In [21]:
# --- PREDICCIONES SOBRE VAL PARA ANÁLISIS DE ERRORES ---
VAL_IMAGES_DIR = YOLO_DATASET_DIR / "images" / "val"
VAL_LABELS_DIR = YOLO_DATASET_DIR / "labels" / "val"

predictions_summary = defaultdict(list)

results = model.predict(
    source=str(VAL_IMAGES_DIR),
    imgsz=cfg["img_size"],
    conf=cfg["conf_threshold"],
    iou=cfg["iou_threshold"],
    save=False,
    verbose=False
)

for r in results:
    img_name = Path(r.path).name
    detected_classes = [model.names[int(c)] for c in r.boxes.cls.tolist()] if r.boxes is not None else []
    predictions_summary[img_name] = detected_classes

len(predictions_summary)


550

In [22]:
# --- ANÁLISIS DE PRESENCIA DE CLASES ---
presence_counter = Counter()

for classes in predictions_summary.values():
    unique = set(classes)
    presence_counter[len(unique)] += 1

print("Cantidad de imágenes según número de clases detectadas:")
for k, v in sorted(presence_counter.items()):
    print(f"{k} clases detectadas:", v)


Cantidad de imágenes según número de clases detectadas:
0 clases detectadas: 64
1 clases detectadas: 291
2 clases detectadas: 185
3 clases detectadas: 10


In [23]:
# --- FRECUENCIA DE CLASES DETECTADAS ---
class_freq = Counter()

for classes in predictions_summary.values():
    for c in classes:
        class_freq[c] += 1

print("Frecuencia de detecciones por clase:")
for k, v in class_freq.items():
    print("-", k, ":", v)


Frecuencia de detecciones por clase:
- car : 1614
- truck : 342
- airplane : 125


## Decisión técnica

Usa estas reglas:

- Si **mAP50 < 0.4** y precision/recall bajos → más datos (dataset completo)
- Si **precision buena pero recall bajo** → bajar conf o entrenar más epochs
- Si **una clase domina y otra casi no aparece** → desbalance de datos


In [25]:
# --- CHECK GPU/CPU (PYTORCH) ---
import torch, platform
print("Python:", platform.python_version())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("GPU:", torch.cuda.get_device_name(0))
    print("VRAM (GB):", round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 2))
else:
    print("Running on CPU")


Python: 3.12.10
Torch: 2.9.1+cpu
CUDA available: False
Running on CPU
